In [5]:
import tensorflow as tf
from tensorflow.contrib import learn 
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib 
import PIL 
from PIL import Image

import numpy as np
from numpy import matlib
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline
import pdb

import glob 

In [13]:
# TRAIN=1200
# VALID=500
TEST=391


scene_dir='/home/jmern91/Documents/Coursework/CS231N/Project_Data_2/flyUpdate/scene/test'
depth_dir='/home/jmern91/Documents/Coursework/CS231N/Project_Data_2/flyUpdate/depth/test'

# X_train = np.zeros([TRAIN, 245, 437, 3])
# Y_train = np.zeros([TRAIN, 245, 437, 3])
# X_valid = np.zeros([VALID, 245, 437, 3])
# Y_valid = np.zeros([VALID, 245, 437, 3])
X_test = np.zeros([TEST, 245, 437, 3])
Y_test = np.zeros([TEST, 245, 437, 3])

# for i in range(TRAIN):
#     idc = idx[i]
#     x_path = scene_dir+str(idc)+'.jpg'
#     y_path = depth_dir+str(idc)+'.jpg'
# #     test = np.array(Image.open(x_path).convert('L'))
# #     pdb.set_trace()
#     X_train[i,:,:,:] = np.array(Image.open(x_path))/255.0
#     Y_train[i,:,:,:] = np.array(Image.open(y_path))/255.0
    
# for i in range(VALID):
#     idc = idx[i+TRAIN]
#     x_path = scene_dir+str(idc)+'.jpg'
#     y_path = depth_dir+str(idc)+'.jpg'
#     X_valid[i,:,:,:] = np.array(Image.open(x_path))/255.0
#     Y_valid[i,:,:,:] = np.array(Image.open(y_path))/255.0

file_list = []
i = 0
for filename in glob.glob('/home/jmern91/Documents/Coursework/CS231N/Project_Data_2/flyUpdate/scene/test/*'): #assuming gif
    im=Image.open(filename)
    X_test[i,:,:,:] = np.array(im)/255.0
    i += 1
    file_list.append(filename)
i = 0

for filename in glob.glob('/home/jmern91/Documents/Coursework/CS231N/Project_Data_2/flyUpdate/depth/test/*'): #assuming gif
    im=Image.open(filename)
#     pdb.set_trace()
    if np.array(im).ndim == 3:
        Y_test[i,:,:,:] = np.array(im)/255.0
    else:
        Y_test[i,:,:,0] = np.array(im)/255.0
        Y_test[i,:,:,1] = np.array(im)/255.0
        Y_test[i,:,:,2] = np.array(im)/255.0
    i += 1
    
# for i in range(TEST):
#     idc = i
#     x_path = scene_dir+str(idc)+'.jpg'
#     y_path = depth_dir+str(idc)+'.jpg'
#     X_test[i,:,:,:] = np.array(Image.open(x_path))/255.0
#     Y_test[i,:,:,:] = np.array(Image.open(y_path))/255.0

# print('Training Data Shape: ', X_train.shape)
# print('Validation Data Shape: ', X_valid.shape)
print('Test Data Shape: ', X_test.shape)

Test Data Shape:  (391, 245, 437, 3)


In [8]:
def run_model(session, loss_val, Xd, Yd, 
              epochs=1, batch_size=64, print_every=100,
              L_rate=None, decay=None,
              training=None, plot_losses=False,save=False):
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    # setting up variables we want to compute (and optimizing)
    # if we have a training function, add that to things we compute
    variables = [loss_val]
    if training_now:
        variables.append(training)
#     pdb.set_trace()
    # counter 
    iter_cnt = 0
    for e in range(epochs):
        # keep track of losses
        losses = []
        # Decay learning rate
        if decay is not None: 
            L_rate *= decay
        
        # make sure we iterate over the dataset once
        for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
            # generate indicies for the batch
            start_idx = (i*batch_size)%X_train.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            # create a feed dictionary for this batch
            feed_dict = {X: Xd[idx,:],
                         Y: Yd[idx,:],
                         is_training: training_now,
                         LR: L_rate}
            # get batch size
            actual_batch_size = Yd[i:i+batch_size].shape[0]
            
            # have tensorflow compute loss and correct predictions
            # and (if given) perform a training step
            if training_now:
                loss, _ = session.run(variables,feed_dict=feed_dict)
            else: 
                loss = session.run(variables,feed_dict=feed_dict)
            # aggregate performance stats
#             pdb.set_trace()
            losses.append(loss*actual_batch_size)
            
            # print every now and then
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration %r: with minibatch training loss = %r " % (iter_cnt,loss))
            iter_cnt += 1
#         pdb.set_trace()
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {1}, Overall loss = {0:.3g}"\
              .format(total_loss,e+1))
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss


In [9]:
def encoder_model(X, is_training):
    conv0 = tf.layers.conv2d(
            inputs=X,
            filters = 8,
            kernel_size = [4,4],
            strides = 2,
            padding = 'valid',
            activation = tf.nn.relu,
#                 name='conv0'
    )

    bn0 = tf.layers.batch_normalization(
            conv0, training=is_training,
#                 name='bn0'
    )

    conv1 = tf.layers.conv2d(
            inputs=bn0,
            filters = 16,
            kernel_size = [3,3],
            strides = 2,
            padding = 'valid',
            activation = tf.nn.relu,
#                 name='conv1'
    )

    bn1 = tf.layers.batch_normalization(
            conv1, training=is_training,
#                 name='bn1'
    )

    conv2 = tf.layers.conv2d(
            inputs=bn1,
            filters = 16,
            kernel_size = [3,3],
            strides = 1,
            padding = 'valid',
            activation = tf.nn.relu,
#                 name='conv2'
    )

    bn2 = tf.layers.batch_normalization(
            conv2, training=is_training,
#                 name='bn2'
    )

    conv3 = tf.layers.conv2d(
            inputs=bn2,
            filters = 24,
            kernel_size = [2,2],
            strides = 1,
            padding = 'valid',
            activation = tf.nn.relu,
#                 name='conv3'
    )
#     print(conv3.shape)
    return conv3


In [10]:
def decoder_model(X, is_training):    
    deconv4 = tf.layers.conv2d_transpose(
            inputs=X,
            filters = 64,
            kernel_size=[6,6],
            strides = 2,
#             padding='same',
            activation=tf.nn.relu)
    
    bn3d = tf.layers.batch_normalization(
            deconv4, training=is_training)
    
    deconv3 = tf.layers.conv2d_transpose(
            inputs=bn3d,
            filters = 32,
            kernel_size = [6,6],
            strides = 2,
#             padding = 'same',
            activation = tf.nn.relu)
    bn2d = tf.layers.batch_normalization(
            deconv3, training=is_training)
    
    deconv2 = tf.layers.conv2d_transpose(
            inputs=bn2d,
            filters = 16,
            kernel_size = [5,5],
            strides = 1,
#             padding = 'same',
            activation = tf.nn.relu)
    bn1d = tf.layers.batch_normalization(
            deconv2, training=is_training)
    
    deconv1 = tf.layers.conv2d_transpose(
            inputs=bn1d,
            filters = 4,
            kernel_size = [5,5],
            strides = 2,
#             padding = 'same',
            activation = tf.nn.relu)
    bn0d = tf.layers.batch_normalization(
            deconv1, training=is_training)
#     print(deconv1.shape)
    img_ = tf.layers.conv2d(
            inputs=bn0d,
            filters = 1,
            kernel_size = [2,2],
            strides = 2,
#             padding = 'same',
            activation = tf.nn.relu)
    img = tf.squeeze(img_)
#     print(img.shape)
    return img


In [11]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 245, 437, 3])
Y = tf.placeholder(tf.float32, [None, 245, 437, 3])
LR = tf.placeholder(tf.float32)
is_training = tf.placeholder(tf.bool)


with tf.variable_scope('Auto_Encoder')  as a_enc:
    y_feats = encoder_model(Y, is_training)
    
with tf.variable_scope('Auto_Decoder') as a_dec:
    y_img = decoder_model(y_feats, is_training)
    
with tf.variable_scope('Trans_Encoder')  as t_enc:
    trans_feats_x = encoder_model(X, is_training)
    
# with tf.variable_scope(t_enc, reuse=True):
#     trans_feats_y = encoder_model(Y, is_training)
    
with tf.variable_scope(a_dec,reuse=True):
    trans_img = decoder_model(trans_feats_x, is_training)

loss = tf.nn.l2_loss(y_img-tf.reduce_mean(Y,axis=3))
mean_loss = tf.reduce_mean(loss) 

trans_loss = tf.nn.l2_loss(trans_feats_x - y_feats)
trans_mean_loss = tf.reduce_mean(trans_loss) 
    
optimizer = tf.train.AdamOptimizer(learning_rate=LR)
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
a_enc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='Auto_Encoder')
a_dec_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='Auto_Decoder')
t_enc_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope='Trans_Encoder')
with tf.control_dependencies(extra_update_ops):
    train_step = optimizer.minimize(mean_loss, var_list=a_enc_vars+a_dec_vars)
    train_step_trans = optimizer.minimize(trans_mean_loss, var_list=t_enc_vars)

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(var_list=t_enc_vars+a_enc_vars+a_dec_vars)
saver.restore(sess, 'trans2_weights/trans2_weights')

In [14]:
for i in range(TEST):
    y = sess.run(trans_img,feed_dict={X: X_test[i:i+1,:,:,:], 
                                     is_training:False})
#     pdb.set_trace()
    result = Image.fromarray((y * 255.0).astype(np.uint8))
    result.save(file_list[i])

In [13]:
y = sess.run(trans_img,feed_dict={X: X_test[i:i+1,:,:,:], 
                                  is_training:False})